# Biodiversity Intactness Index change in Phoenix, AZ

Author: Josephine Cardelle

[Github repository](https://github.com/jocardelle/biodiversity-intactness-index)

## About

### Purpose

### Highlights

### About the data

### References

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr

from pystac_client import Client  # To access STAC catalogs

import planetary_computer  # To sign items from the MPC STAC catalog 

from IPython.display import Image  # To nicely display images

In [6]:
# Access MPC catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)